# Save, Load and Export Models in Keras

## Task 1: Import Libraries

In [1]:
import tensorflow as tf
import numpy as np
import os

print('This notebook works with TensorFlow version:', tf.__version__)

folders = ['tmp', 'models', 'model_name', 'weights']
for folder in folders:
    if not os.path.isdir(folder):
        os.mkdir(folder)

print(os.listdir('.'))

This notebook works with TensorFlow version: 2.0.0-dev20190731
['Save, Load and Export Keras Models - Completed.ipynb', 'weights', '.ipynb_checkpoints', 'model_name', 'models', 'tmp']


## Task 2: Create Model

In [2]:
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


## Task 3: Data Preprocessing

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

x_train = np.reshape(x_train, (x_train.shape[0], 784))/255.
x_test = np.reshape(x_test, (x_test.shape[0], 784))/255.

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

## Task 4: Model Checkpoint During Training

In [4]:
checkpoint_dir = 'weights/'

_ = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=2,
    batch_size=512,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            os.path.join(checkpoint_dir, 'epoch_{epoch:02d}_acc_{val_acc:.4f}'),
            monitor='val_acc', save_weights_only=True, save_best_only=True
        )
    ]
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Apply a constraint manually following the optimizer update step.
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 1s 24us/sample - loss: 0.6726 - acc: 0.7711 - val_loss: 0.5163 - val_acc: 0.8200
Epoch 2/2
60000/60000 [==============================] - 1s 10us/sample - loss: 0.4300 - acc: 0.8499 - val_loss: 0.4365 - val_acc: 0.8439


In [5]:
os.listdir(checkpoint_dir)

['epoch_02_acc_0.8439.index',
 'epoch_01_acc_0.8200.index',
 'epoch_02_acc_0.8439.data-00000-of-00001',
 'checkpoint',
 'epoch_01_acc_0.8200.data-00000-of-00001']

## Task 5: Load Weights

In [6]:
model = create_model()
print(model.evaluate(x_test, y_test, verbose=False))

[2.303709535217285, 0.0515]


In [8]:
model.load_weights('weights/epoch_02_acc_0.8439')
print(model.evaluate(x_test, y_test, verbose=False))

[0.4365476734399796, 0.8439]


## Task 6: Saving Complete Model During Training

In [9]:
models_dir = 'models'

model = create_model()

_ = model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=2,
    batch_size=512,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            os.path.join(models_dir, 'epoch_{epoch:02d}_acc_{val_acc:.4f}.h5'),
            monitor='val_acc', save_weights_only=False, save_best_only=False
        )
    ]
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 1s 12us/sample - loss: 0.7019 - acc: 0.7699 - val_loss: 0.5072 - val_acc: 0.8193
Epoch 2/2
60000/60000 [==============================] - 1s 10us/sample - loss: 0.4425 - acc: 0.8446 - val_loss: 0.4559 - val_acc: 0.8338


In [10]:
os.listdir(models_dir)

['epoch_01_acc_0.8193.h5', 'epoch_02_acc_0.8338.h5']

## Task 7: Load Models

In [11]:
model = create_model()
print(model.evaluate(x_test, y_test, verbose=False))

[2.363083916091919, 0.0357]


In [13]:
model = tf.keras.models.load_model('models/epoch_02_acc_0.8338.h5')
model.summary()
print(model.evaluate(x_test, y_test, verbose=False))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_7 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________
[0.4558810998439789, 0.8338]


## Task 8: Manually Saving Weights and Models

In [14]:
model.save_weights('tmp/manually_saved')
print(os.listdir('tmp'))

['manually_saved.data-00000-of-00001', 'checkpoint', 'manually_saved.index']


In [15]:
model.save('tmp/manually_saved_model.h5')
print(os.listdir('tmp'))

['manually_saved_model.h5', 'manually_saved.data-00000-of-00001', 'checkpoint', 'manually_saved.index']


## Task 9: Exporting and Restoring SavedModel Format

In [16]:
model.save('model_name')
print(os.listdir('model_name'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model_name/assets
['saved_model.pb', 'variables', 'assets']


In [17]:
model = tf.keras.models.load_model('model_name')
print(model.evaluate(x_test, y_test, verbose=False))

[0.4558810998439789, 0.8338]
